## Testing of GPT 117M model

First, follow setup instructions from https://github.com/openai/gpt-2/blob/master/DEVELOPERS.md

The 117M model file is a 475MB download...

In [1]:
import json
import os
import numpy as np
import tensorflow as tf

import model, sample, encoder

In [2]:
# due to paths in function below, expects to run from top level dir of repo
os.chdir('/Users/seth/onedrive/Documents/Development/gpt-2')

In [3]:
model_name='117M'
enc = encoder.get_encoder(model_name)
hparams = model.default_hparams()
with open(os.path.join('models', model_name, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))

In [4]:
def sample_model(
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
):
    """
    Run the sample_model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to
     reproduce results
    :nsamples=0 : Number of samples to return, if 0, continues to
     generate samples indefinately.
    :batch_size=1 : Number of batches (only affects speed/memory).
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
    """
    if length is None:
        length = hparams.n_ctx
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        np.random.seed(seed)
        tf.set_random_seed(seed)

        output = sample.sample_sequence(
            hparams=hparams, length=length,
            start_token=enc.encoder['<|endoftext|>'],
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )[:, 1:]

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
        saver.restore(sess, ckpt)

        generated = 0
        while nsamples == 0 or generated < nsamples:
            out = sess.run(output)
            for i in range(batch_size):
                generated += batch_size
                text = enc.decode(out[i])
                print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                print(text)

In [5]:
def interact_model(
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    prompt=None
):
    """
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
    """
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
        saver.restore(sess, ckpt)

        context_tokens = enc.encode(prompt)
        generated = 0
        for _ in range(nsamples // batch_size):
            out = sess.run(output, feed_dict={
                context: [context_tokens for _ in range(batch_size)]
            })[:, len(context_tokens):]
            for i in range(batch_size):
                generated += 1
                text = enc.decode(out[i])
                print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                print(text)
        print("=" * 80)

In [6]:
%%time
sample_model(seed=42)
# takes about 15 seconds when running with 1 x NVIDIA Tesla T4
# on macbook pro 1:30
# ~ 6 times slower with CPU vs GPU

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
======================================== SAMPLE 1 ========================================
: Checking Technologies to Tear Down Big Banks Investments Ernest Chris Pearl , S. Gelles , Y. Husain , Philip : Cheers. Until I realized they were part of an oligarchy yet they turned down my requests. RIA Novosti : Hey, Eroded, nice profile. Such companies don't want to pay you a penny: Cheers. Until I realised they were part of an oligarchy yet they turned down my requests. Uniprole Press : Such companies don't want to pay you a penny: Guess you said yes? They said no. OrderMark : No much, handle it. Ignoring any outrage about unreasonable paperwork, they said don't

In [7]:
%%time
sample_model(seed=42, top_k=40, temperature=0.7)

INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
======================================== SAMPLE 1 ========================================
I've been using the X1 with my 2 year old for a decade now. It's been really fun to use. I loved the way it works and was really impressed with how easy it was to use. The cable comes with an extra cord so you can plug in whatever you want. The X1 is very sturdy and easy to use. It's not big, but it's not big or heavy. The cable is sturdy. The cable comes with an extra cord so you can plug in whatever you want. The X1 is very sturdy and easy to use. It's not big, but it's not big or heavy. The cable is sturdy. The cable comes with an extra cord so you can plug in whatever you want.

Rated 5 out of 5 by Anonymous from Nice to use This cable is great for any situation. It's easy to use and it's also quick to use. I would recommend this cable to anyone who wants a good, reliable cable that would work with most other cables. I would rec

In [12]:
%%time
interact_model(seed=42, top_k=100, length=183, prompt="""Hi Seth,

I worked off of Tell's corrected letter, much of which I agree with.  The one thing
I can tell you is that if you put the edited text in both the response letter and
in the revision, then that will help the referees.  This is not for typographical
changes but for the substantive changes.  I have indicated in the response letter
where it would be good for you to add in what text you wrote to address/clarify the
comments.


This is a really good paper, and I appreciate your involving us with it.

Best,
Debashis""", temperature=0.75)
# performance highly dependent on parameters
# on 1 x NVIDIA Tesla T4 10 seconds
# on macbook pro 39 seconds

INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
======================================== SAMPLE 1 ========================================



As I said earlier, I think it's fair to say that the error with the

toughness statement appeared in the new content, as is the fact that the original

text was not only too late, but was almost certainly wrong. 


I'll give you the best possible statement of that, as you can probably tell from the

comment comment.  It is not clear until after the correction that the

errors were completely intentional.  I don't know if it was intentional, or if the

criteria were so wrong as to warrant the correction. 


The last thing I would say is that I think this is a good

revision by Jonathan S. Ehrlich, and I would like to thank him for the

review of the new content.  I would also like to thank you for your time.



CPU times: user 20.4 s, sys: 13.2 s, total: 33.5 s
Wall time: 14.6 s


In [20]:
%%time
interact_model(seed=42, top_k=100, length=51, prompt="""Senate Democrats just voted against legislation
to prevent the killing of newborn infant children. The Democrat position on abortion is now so extreme
that they don’t mind executing babies AFTER birth....

....This will be remembered as one of the most shocking votes in the history of Congress. If there is one
thing we should all agree on, it’s protecting the lives of innocent babies.""", temperature=0.5)

INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
======================================== SAMPLE 1 ========================================


...We should never forget that the National Abortion Federation’s position on abortion was that it was a health care measure that should be

protected as a right’ and that the right to life was a fundamental right’


CPU times: user 8.39 s, sys: 1.96 s, total: 10.4 s
Wall time: 7.18 s
